In [30]:
import os
import numpy as np
import pandas as pd
import re
import time
#import torch
import transformers
import emoji

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from torch.utils.data import Dataset
from transformers import (ElectraForSequenceClassification, ElectraTokenizerFast,
                          InputFeatures, Trainer, TrainingArguments)

In [ ]:
a = time.time()
data = pd.read_csv('depresion twitter.csv',
                   dtype={'Unnamed: 0': int, 'text': str, 'label': bool, 'user': str}, 
                   sep = ',').drop_duplicates()

#renombro columnas y elimino las Id
data = data.rename(columns={'text': 'texto', 'label': 'depresion', 'user': 'usuario'})
data = data.drop('Unnamed: 0', axis = 1)

#cambio los usuarios asignando un valor numérico y extiendo el conteo a los Falsos (antes volvía a llamarlos 'user0' y contaba de nuevo desde ahí)
data['usuario'] = data['usuario'].apply(lambda x: x[-len(x) + 4:])
data['usuario'] = data['usuario'].astype(int)
data['usuario'][data['depresion'] == False] = data['usuario'] + 1043
print('Cargar los datos: {}s'.format(round(time.time() - a, 2)))
data.head()

In [ ]:
a = time.time()
#salvemos las ñ's, pero normalicemos lo demás
data['texto'] = data['texto'].apply(lambda x: x.replace('ñ', '$n$'))
print('ñ --> $n$')
data['texto'] = data['texto'].str.normalize('NFKD').str.normalize('NFKD').str.lower().str.encode('ascii', errors = 'ignore').str.decode('utf-8')
print('asci')
data['texto'] = data['texto'].apply(lambda x: x.replace('$n$', 'ñ'))
print('$n$ --> ñ')
print('Normalización: {}s'.format(round((time.time() - a),2)))
#data = data.drop('texto_original', axis = 1) 
data.head()

In [ ]:
#Eliminamos los hagstags
data['texto'] = data['texto'].apply(lambda x: re.sub(r'#(\w+)', '', x))
#eliminamos las menciones
data['texto'] = data['texto'].apply(lambda x: re.sub(r'@(\w+)', '', x))
#eliminamos los enlaces a páginas web
url_regex = r'(https?:\/\/)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)'

data['texto'] = data['texto'].apply(lambda x: re.sub(url_regex, '', x))

In [ ]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats1
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [ ]:
#vamos a eliminar los caracteres repetidos infinitamente y los números.
caract_rep = r'/([a-zA-Z])\1{3,}/gi'

data['texto'] = data['texto'].apply(lambda x: re.sub(caract_rep, '', x))

numeros = r'/([0])\1{3,}/gi'

data['texto'] = data['texto'].apply(lambda x: re.sub(caract_rep, '', x))

In [ ]:
def remove_URL(headline_text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', headline_text)

data['texto'] = data['texto'].apply(remove_URL)

In [ ]:
D = data[data['depresion'] == 1].sample(100000)
ND = data[data['depresion'] == 0].sample(100000)

data = pd.concat([D, ND], axis = 0)

del D, ND

In [ ]:
data

In [29]:
data.to_csv('limpio.csv')